In [9]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import re
import math
from collections import Counter
from transformers import pipeline

In [4]:
first_url = 'https://www.glassdoor.com/Reviews/Revolut-Reviews-E1176471.htm?filter.iso3Language=eng'
pwd = os.getcwd()
driver_path = pwd + "/chromedriver.exe"
wd = webdriver.Chrome(executable_path=driver_path)
wd.get(first_url)

soup = bs(wd.page_source, 'html.parser')
total_reviews = re.findall(r"\d+[^\s]", soup.find(class_='css-ntu5m7 col-6 my-0').text.strip())[0]

wd.quit()

C:\Users\u10054206\AppData\Local\Temp\ipykernel_22396\2413263014.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=driver_path)


In [5]:
total_pages = math.ceil(int(total_reviews) / 10)
total_pages

89

In [12]:
ratings = []
employee_status = []
review_titles = []
review_dates = []
review_pros = []
review_cons = []
num_people_helpful = []
for i in range(total_pages):
    print(i)
    page_number = str(i)
    url = f'https://www.glassdoor.com/Reviews/Revolut-Reviews-E1176471_P{page_number}.htm?filter.iso3Language=eng'
    pwd = os.getcwd()
    driver_path = pwd + "/chromedriver.exe"

    # setup chrome to run headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")
    
    wd = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

    wd.get(url)

    time.sleep(2)

    soup = bs(wd.page_source, 'html.parser')

    for review in soup.find_all(class_='gdReview'):
        ratings.append(review.find(class_='ratingNumber mr-xsm').text.strip())

        employee_status.append(review.find(class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0').text.strip())

        review_titles.append(review.find(class_='mb-xxsm mt-0 css-93svrw el6ke055').text.strip())

        review_dates.append(review.find(class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey').text.strip())

        review_pros.append(review.find("span", {"data-test": "pros"}).text.strip().replace("\r", "").replace("\n", ""))

        review_cons.append(review.find("span", {"data-test": "cons"}).text.strip().replace("\r", "").replace("\n", ""))

        try:
            num_people_helpful.append(re.findall(r"\d+", review.find(class_="common__EiReviewDetailsStyle__socialHelpfulcontainer pt-std").text.strip())[0].replace("\xa0", ""))
        except IndexError as e:
            num_people_helpful.append(0)
    wd.quit()

0


C:\Users\u10054206\AppData\Local\Temp\ipykernel_22396\3559197956.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=driver_path, options=chrome_options)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [13]:
df = pd.DataFrame(data={"Rating":ratings, "Employee Status":employee_status, "Review Title":review_titles, "Date":review_dates,
                    "Pros":review_pros, "Cons":review_cons, "Num People Helpful":num_people_helpful})
df

,Rating,Employee Status,Review Title,Date,Pros,Cons,Num People Helpful
0,5.0,Current Employee,Review,"Apr 26, 2022 - Anonymous Employee",Learning new content. Extremely detailed train...,Amount of initial information can be overwhelm...,2
1,1.0,"Current Employee, less than 1 year",Don't Believe What They Tell You,"Jan 31, 2022 - Growth",- Competitive comp - Flat org provides a lot o...,- No job security or work/life balance - No co...,41
2,1.0,"Former Employee, less than 1 year",misunderstanding of the market,"Mar 8, 2022 - Remote Sales Representative","remote work, offers health insurance day 1",does not understand the market in the U.S. The...,11
3,2.0,"Former Employee, more than 1 year",Terrible Ideologies,"Mar 2, 2021 - Senior Recruiter","-Worked with smart and passionate people, a bu...",-The executive leadership team at HQ are deep ...,84
4,4.0,"Current Employee, more than 1 year",Great Place to Work,"Mar 2, 2022 - Senior Software Engineer",- Huge growth opportunities to tech leadership...,- Startups environment: a lot of things change...,1
...,...,...,...,...,...,...,...
885,3.0,Current Employee,Exciting growth but bad culture,"Jan 5, 2018 - Anonymous Employee","Growth was really fast, fun team, cool product.","Bad working culture, staff not really respected.",4
886,1.0,Former Employee,n/a,"Feb 9, 2018 - Anonymous Employee",Opportunity to be creative and work on projects.,Worst company culture. No recognition. Bullyin...,16
887,1.0,"Current Employee, less than 1 year",People too stressed to relax,"Mar 21, 2019 - Anonymous Employee",Company has a good reputation outside of itsel...,People are much too busy and arrogant to even ...,8
888,1.0,Current Employee,Worst company,"Apr 27, 2018 - Anonymous Employee",Could not find anything good to say,Worst company ever.I will definitely not recom...,19


In [14]:
df.to_csv('reviews.csv', sep=';', index=False)

In [15]:
classifier = pipeline('sentiment-analysis')

In [16]:
classifier('We are ver happy to introduce pipeline to the transformers repository.')

[{'label': 'POSITIVE', 'score': 0.9989639520645142}]

In [17]:
df_import = pd.read_csv('reviews.csv', sep=';')
df_import

,Rating,Employee Status,Review Title,Date,Pros,Cons,Num People Helpful
0,5.0,Current Employee,Review,"Apr 26, 2022 - Anonymous Employee",Learning new content. Extremely detailed train...,Amount of initial information can be overwhelm...,2
1,1.0,"Current Employee, less than 1 year",Don't Believe What They Tell You,"Jan 31, 2022 - Growth",- Competitive comp - Flat org provides a lot o...,- No job security or work/life balance - No co...,41
2,1.0,"Former Employee, less than 1 year",misunderstanding of the market,"Mar 8, 2022 - Remote Sales Representative","remote work, offers health insurance day 1",does not understand the market in the U.S. The...,11
3,2.0,"Former Employee, more than 1 year",Terrible Ideologies,"Mar 2, 2021 - Senior Recruiter","-Worked with smart and passionate people, a bu...",-The executive leadership team at HQ are deep ...,84
4,4.0,"Current Employee, more than 1 year",Great Place to Work,"Mar 2, 2022 - Senior Software Engineer",- Huge growth opportunities to tech leadership...,- Startups environment: a lot of things change...,1
...,...,...,...,...,...,...,...
885,3.0,Current Employee,Exciting growth but bad culture,"Jan 5, 2018 - Anonymous Employee","Growth was really fast, fun team, cool product.","Bad working culture, staff not really respected.",4
886,1.0,Former Employee,NaN,"Feb 9, 2018 - Anonymous Employee",Opportunity to be creative and work on projects.,Worst company culture. No recognition. Bullyin...,16
887,1.0,"Current Employee, less than 1 year",People too stressed to relax,"Mar 21, 2019 - Anonymous Employee",Company has a good reputation outside of itsel...,People are much too busy and arrogant to even ...,8
888,1.0,Current Employee,Worst company,"Apr 27, 2018 - Anonymous Employee",Could not find anything good to say,Worst company ever.I will definitely not recom...,19


### Convert text into tokens

In [18]:
# tokenize the text with regular expressions
# "w+": This expression matches the alphanumeric character in the text
import re
token_positive = []
token_negative = []

for i in range(len(df_import['Pros'])):
    token_positive.append(re.findall('\w+', df_import['Pros'][i]))

for i in range(len(df_import['Cons'])):
    token_negative.append(re.findall('\w+', df_import['Cons'][i]))

token_negative[:10]

[['Amount',
  'of',
  'initial',
  'information',
  'can',
  'be',
  'overwhelming',
  'at',
  'first'],
 ['No',
  'job',
  'security',
  'or',
  'work',
  'life',
  'balance',
  'No',
  'company',
  'culture',
  'due',
  'to',
  'high',
  'turnover',
  'Management',
  'doesn',
  't',
  'value',
  'people',
  'Blatant',
  'bias',
  'when',
  'it',
  'comes',
  'to',
  'diversity'],
 ['does',
  'not',
  'understand',
  'the',
  'market',
  'in',
  'the',
  'U',
  'S',
  'The',
  'commission',
  'rate',
  'is',
  'perhaps',
  'the',
  'worst',
  'in',
  'the',
  'industry',
  'the',
  'company',
  'seems',
  'to',
  'still',
  'be',
  'trying',
  'to',
  'figure',
  'out',
  'why',
  'the',
  'U',
  'S',
  'Does',
  'not',
  'convert',
  'as',
  'regularly',
  'to',
  'other',
  'currency',
  'The',
  'strategic',
  'plan',
  'is',
  'non',
  'existent',
  'Along',
  'with',
  'this',
  'they',
  'spy',
  'on',
  'employees'],
 ['The',
  'executive',
  'leadership',
  'team',
  'at',
  '

### Convert tokens into Words

In [19]:
words_positive = []
words_negative = []

for i in range(len(token_positive)):
    for word in token_positive[i]:
        words_positive.append(word.lower())

for i in range(len(token_negative)):
    for word in token_negative[i]:
        words_negative.append(word.lower())

words_negative[:100]

['amount',
 'of',
 'initial',
 'information',
 'can',
 'be',
 'overwhelming',
 'at',
 'first',
 'no',
 'job',
 'security',
 'or',
 'work',
 'life',
 'balance',
 'no',
 'company',
 'culture',
 'due',
 'to',
 'high',
 'turnover',
 'management',
 'doesn',
 't',
 'value',
 'people',
 'blatant',
 'bias',
 'when',
 'it',
 'comes',
 'to',
 'diversity',
 'does',
 'not',
 'understand',
 'the',
 'market',
 'in',
 'the',
 'u',
 's',
 'the',
 'commission',
 'rate',
 'is',
 'perhaps',
 'the',
 'worst',
 'in',
 'the',
 'industry',
 'the',
 'company',
 'seems',
 'to',
 'still',
 'be',
 'trying',
 'to',
 'figure',
 'out',
 'why',
 'the',
 'u',
 's',
 'does',
 'not',
 'convert',
 'as',
 'regularly',
 'to',
 'other',
 'currency',
 'the',
 'strategic',
 'plan',
 'is',
 'non',
 'existent',
 'along',
 'with',
 'this',
 'they',
 'spy',
 'on',
 'employees',
 'the',
 'executive',
 'leadership',
 'team',
 'at',
 'hq',
 'are',
 'deep',
 'into',
 'the',
 'weeds']

### Remove all stopwords

In [20]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u10054206\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
sw = nltk.corpus.stopwords.words('english')
sw[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

### Get a list of words without stop words

In [22]:
words_ne_positive = []
words_ne_negative = []

for word in words_positive:
    if word not in sw:
        words_ne_positive.append(word)

for word in words_negative:
    if word not in sw:
        words_ne_negative.append(word)

#### Count number of words grouped by positive and negative reviews

In [23]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

for word in words_ne_positive:
    positive_counts[word]+=1
    total_counts[word]+=1

for word in words_ne_negative:
        negative_counts[word]+=1
        total_counts[word]+=1

positive_counts.most_common()[0:10]

[('work', 302),
 ('people', 268),
 ('great', 251),
 ('good', 243),
 ('company', 204),
 ('team', 130),
 ('lot', 121),
 ('product', 111),
 ('working', 103),
 ('fast', 101)]

In [24]:
def get_words_count(dictionary):
    keys = []
    values = []
    for key, value in dictionary.items():
        keys.append(key)
        values.append(value)
    return keys, values

positive_counts_keys, positive_counts_values = get_words_count(positive_counts)
negative_counts_keys, negative_counts_values = get_words_count(negative_counts)
total_counts_keys, total_counts_values = get_words_count(total_counts)

In [25]:
df_positive_words = pd.DataFrame(data={"Word":positive_counts_keys, "Count":positive_counts_values, "Sentiment Label":"POSITIVE"})
df_negative_words = pd.DataFrame(data={"Word":negative_counts_keys, "Count":negative_counts_values, "Sentiment Label":"NEGATIVE"})
df_total_words = pd.DataFrame(data={"Word":total_counts_keys, "Count":total_counts_values, "Sentiment Label":"TOTAL"})

total_words = [df_positive_words, df_negative_words, df_total_words]
total_words

df_words_final = pd.concat(total_words)
df_words_final

,Word,Count,Sentiment Label
0,learning,47,POSITIVE
1,new,61,POSITIVE
2,content,3,POSITIVE
3,extremely,29,POSITIVE
4,detailed,2,POSITIVE
...,...,...,...
4402,strategyunorganised,1,TOTAL
4403,managementlack,1,TOTAL
4404,directionarrogant,1,TOTAL
4405,coounprofessionaliam,1,TOTAL


In [26]:
df_words_final.to_csv('reviews_words.csv', index=False, sep=';')